In [1]:
using LightGraphs

"""
inverse logit function
"""
invLogit(x) = 1./(1.+e.^-x)   

"""
given graph and probability, adds a node which must have >0
connections by flipping biased coin for each existing node
"""
function addNode2(graph, p)
    add_vertex!(graph)
    x = nv(graph)
    degree = 0
    while degree ==0
        flips = rand(x-1)
        for i = 1:x-1
            if p[i]>flips[i]
                add_edge!(graph,i,x)
                degree +=1 
            end
        end
    end
    return graph
end


"""
given graph, b vector, and a_0, adds a new node as specifiec by the model
"""
function addPrefNode(g,b,a_0 = -7)
    n = nv(g)
    L = laplacian_matrix(g)
    a = lufact(L) \ (b - mean(b))    
    p = invLogit(a+a_0)
    addNode2(g,p)
    push!(b,0)
    return g
end


"""
given graph and number of new edges desired, randomly adds edges between existing nodes
"""
function randEdgeGen(graph, newedges)
    for i in 1:newedges
        z = newedges
        x = collect(1:nv(graph))
        edge1 = rand(x)
        deleteat!(x, edge1)
        edge2 = rand(x)
        add_edge!(graph,edge1,edge2)
    end
    return graph
end
;

In [2]:
"""
soft threshold
"""
soft(c,lambda) = sign(c).*max(abs(c)-lambda/2,0)

"""
computes gradient
"""
function gradient2(a,a_0,u,L,rho,b,y)
    grad = -1.*(y-invLogit(a+a_0))+L*u + rho*L*(L*a-b)
    return grad
end;


"""
computes hessian
"""
function hessian(a,a_0,rho,L)
    hess = Diagonal(vec((invLogit(a+a_0).*(1-invLogit(a+a_0)))))+rho*L^2
    return hess
end;




"""
newton raphson for a update
"""
function newton(y_i,a_0,L,rho,b,u)
    a = zeros(length(y_i),1)
    a_old = a
    iters = 0
    diff = 1.0
    while(diff >STOP_DIFF && iters< MAX_ITER )
        grad = gradient2(a_old,a_0,u,L,rho,b,y_i)
        hess = hessian(a_old,a_0, rho,L)
        a = a_old - inv(hess)*grad
        diff = norm(a-a_old)
        a_old = a
        iters = iters+1
    end
    if(iters == MAX_ITER)
        print("max iter reached")
    end
    return a
end
;




In [3]:
levels = 10     #number of levels in binary tree
g = BinaryTree(levels)
n = nv(g)
b = (rand(n) .< 8 / n)*1. 
genb = copy(b)  # save for later
g = randEdgeGen(g,1000)
L =  laplacian_matrix(g)
numnewnodes = 1
a_0 = -5
g = addPrefNode(g,b, a_0)
connects = zeros(2^levels-1,1)  #-1 for -1 1 coding
connects[neighbors(g,nv(g))] = 1
A = connects


const MAX_ITER = 500
const STOP_DIFF = 0.001;


t = 2^levels-1+numnewnodes #number of nodes at time t
t_0 = 2^levels-1  # number of initial nodes
;


rho = 1.5
lambda = 1.1
new = numnewnodes
a = zeros(t-1,new)
b = zeros(t_0)
u = zeros(t-1,new)
#	alpha = 1.5  #relaxation parameter
iters = 0
diff = 1.0
b_old = b;

In [6]:
for j in 1:200     
    a= newton(A,a_0,L,rho,b,u)
    #b update
    c = zeros(t-1)
    c = c+ u+rho*(L*a)/(rho)
    b = soft(c,lambda)
    u = u+ rho*(L*a-b)
    diff  = norm(b-b_old)
    b_old = b
    println(diff)
end

0.005442271796196595
0.005275152120191739
0.005207523468982031
0.005153642274412673
0.005016017032537633
0.004948793833428739
0.004897970396335478
0.004855200340167594
0.004821658105315304
0.004714769086804971
0.004670578291933103
0.004656000490913556
0.00464616841428002
0.004559106164780534
0.004534417519233187
0.004514713836787275
0.004187771628514748
0.004114968269024912
0.003937028356250598
0.0039168187588222185
0.003897838527850984
0.003878821559793315
0.0038595741453125733
0.003839392724205656
0.003699699794310509
0.00367990745164503
0.0036608623246184822
0.003642107255019426
0.003623658723796826
0.0036055341144310186
0.003587744506962438
0.0035702925686276286
0.0035531719281487995
0.0035363679274503525
0.0035198595186753993
0.0034623333344297896
0.003404213203382707
0.0033886064247254924
0.003372935705291712
0.0033572940653941365
0.0033417780516274928
0.003326493466278623
0.003311541475512503
0.003234047828349371
0.0032132783001809656
0.003198943753794504
0.0031853013341812293
0

In [5]:
sort(b[:,1])

1023-element Array{Float64,1}:
 -0.245484
 -0.20288 
 -0.167815
 -0.163646
 -0.131172
 -0.128876
 -0.128876
 -0.119823
 -0.117269
 -0.117269
 -0.114223
 -0.112334
 -0.100942
  ⋮       
 -0.0     
 -0.0     
 -0.0     
 -0.0     
 -0.0     
 -0.0     
 -0.0     
 -0.0     
 -0.0     
 -0.0     
  0.0     
  3.19736 